# RKI COVID-19 Stats 

Data provided by Robert Koch Institut (RKI) and processed by [NPGEO](https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0).
Data License: Robert Koch-Institut (RKI), [dl-de/by-2-0](https://www.govdata.de/dl-de/by-2-0).

Source code to generate this website is available on [GitHub](https://github.com/woefe/covid19stats).

In [ ]:
import datetime
from pathlib import Path

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

figsize=(20,13)
pd.options.display.max_rows = 1000
SILO_DIR = Path("./autodata/covid19stats-autodata/data/silo")

def prepare(col: str, refcol: str, df: pd.DataFrame) -> pd.Series:
    """Generate a series that can be use with the standar sum() function
    
    The documentation of the dataset specifies that "AnzahlFall" and other
    columns should not be simply summed up.
    """
    # Uses numpy's vectorize() for massive performance improvements.
    # Does the same thing as:
    # return pd.Series((row[col] if row[refcol] in [0,1] else 0) for _, row in df.iterrows())
    
    def preparation(col, refcol):
        return col if refcol == 0 or refcol == 1 else 0
    prep = np.vectorize(preparation)
    return pd.Series(prep(df[col].to_numpy(), df[refcol].to_numpy()))
    
def get_cases(date: datetime.date, url_prefix) -> pd.DataFrame:
    date_str = date.isoformat()
    rki_cases = pd.read_csv(f"{url_prefix}/{date_str}.csv")
    rki_cases["Datenstand"] = pd.to_datetime(rki_cases["Datenstand"], format="%d.%m.%Y, %H:%M Uhr")
    rki_cases["Meldedatum"] = pd.to_datetime(rki_cases["Meldedatum"], format="%Y/%m/%d %H:%M:%S")
    rki_cases["Refdatum"] = pd.to_datetime(rki_cases["Refdatum"], format="%Y/%m/%d %H:%M:%S")
    rki_cases["cases"] = prepare("AnzahlFall", "NeuerFall", rki_cases)
    rki_cases["deaths"] = prepare("AnzahlTodesfall", "NeuerTodesfall", rki_cases)
    rki_cases["recovered"] = prepare("AnzahlGenesen", "NeuGenesen", rki_cases)
    return rki_cases

# Main data source
rki_cases = get_cases(datetime.date.today(), url_prefix=str(SILO_DIR))

# Per district data
rki_lkr = pd.read_csv("data/RKI_Corona_Landkreise.csv")

Last update:

In [ ]:
print(datetime.datetime.now().strftime("%c"))

## Overview

In [ ]:
# Overview over entire data. Sanity check. Should be the same data as on the RKI dashboard
overall = rki_cases[["cases", "deaths", "recovered"]].sum(axis="rows")
print(f"          Overall cases: {overall.cases}")
print(f"         Overall deaths: {overall.deaths}")
print(f"Overall recovered cases: {overall.recovered}")
print(f" Currently active cases: {overall.cases - overall.deaths - overall.recovered}")
print(f"      Overall deathrate: {round(overall.deaths / overall.cases * 100, 3)}%")

## Cases and deathrate by age (all of Germany, since January 2020)

In [ ]:
by_age = rki_cases.groupby("Altersgruppe").sum()[["cases", "deaths", "recovered"]]
by_age["sick"] = by_age.cases - by_age.deaths - by_age.recovered
by_age["deathrate %"] = (by_age.deaths / by_age.cases) * 100
by_age

## Cumulative cases since January 2020


In [ ]:
#datecol = "Refdatum"
datecol = "Meldedatum"

#per_day = rki_cases[rki_cases["Bundesland"] == "Bayern"].copy()
per_day = rki_cases.copy()

per_day[datecol] = rki_cases[datecol].dt.floor("D")
per_day = per_day.groupby(datecol).sum()[["cases", "deaths", "recovered"]]
per_day.sort_values(datecol, inplace=True)
per_day["cum_cases"] = per_day["cases"].cumsum()
per_day["cum_deaths"] = per_day["deaths"].cumsum()
per_day["cum_recovered"] = per_day["recovered"].cumsum()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
import matplotlib.dates as mdates

plt.plot(per_day.index, per_day.cum_cases, label="Cases")
plt.plot(per_day.index, per_day.cum_deaths, label="Deaths")

# The recovered cases should "lag behind" by approx., but it doesn't.
# That's because the dataset does not have a date indicating when a patient has recovered.
# Instead the original record seems to be updated.
# Hence, the I won't plot the recovered cases.
#plt.plot(per_day.index, per_day.cum_recovered, label="Recovered")
ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_minor_locator(mdates.DayLocator(interval=1))

fig.autofmt_xdate()
plt.legend()
plt.title("Cumulative cases (all of Germany)")
plt.grid()
plt.show()


## Daily registered cases per state

In [ ]:
states = list(rki_cases.drop_duplicates("Bundesland").Bundesland)

colors = ["black", "gray", "lightgray", "red", "lightcoral", "goldenrod", "yellow", "olive", "chartreuse", "forestgreen", "aqua", "powderblue", "royalblue", "darkviolet", "violet", "fuchsia"]

#datecol = "Refdatum"
datecol = "Meldedatum"

def plot_data(state):
    df = rki_cases[rki_cases["Bundesland"] == state].copy()

    df[datecol] = rki_cases[datecol].dt.floor("D")
    df = df.groupby(datecol).sum()
    df.sort_values(datecol, inplace=True)
    return df


by_state = pd.DataFrame(index=rki_cases.groupby(datecol).first().index)
for state in states:
    by_state[state] = plot_data(state).cases
by_state = by_state.fillna(0) 

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

below = None
for state, color in zip(states, colors):
    df = by_state[state]
    if below is None:
        plt.bar(by_state.index, df, label=state, color=color)
        below = df
    else:
        plt.bar(by_state.index, df, label=state, color=color, bottom=below)
        below = below.add(df, fill_value=0)
        

ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_minor_locator(mdates.DayLocator(interval=1))

fig.autofmt_xdate()

plt.legend()
plt.title("Registered cases stacked")
plt.grid()
plt.show()

## Estimated active cases over time

In [ ]:
import calendar
import os
import logging
from multiprocessing import Pool
from typing import Tuple

def get_active_cases(date: datetime.date) -> Tuple[datetime.date,int]:
    date_str = date.isoformat()
    try:
        data_file = SILO_DIR / f"{date_str}.csv"
        rki_cases = pd.read_csv(data_file)
    except:
        #logging.warning("Ignoring file %s", data_file)
        return date, -1
    cases = prepare("AnzahlFall", "NeuerFall", rki_cases).sum()
    deaths = prepare("AnzahlTodesfall", "NeuerTodesfall", rki_cases).sum()
    recovered = prepare("AnzahlGenesen", "NeuGenesen", rki_cases).sum()
    return date, cases - deaths - recovered

cal = calendar.Calendar()
dates = (
    date 
    for month in range(4,11)
    for date in cal.itermonthdates(2020, month)
    if date <= datetime.date.today()
)

with Pool(len(os.sched_getaffinity(0))) as pool:
    cases_per_day = pool.map(get_active_cases, dates)

In [ ]:
active_daily = pd.DataFrame(cases_per_day, columns=["date", "active_cases"]).sort_values("date")
fig, ax = plt.subplots(figsize=figsize)

plt.plot(active_daily.date, active_daily.active_cases, label="Estimated active cases")

ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_minor_locator(mdates.DayLocator(interval=1))

fig.autofmt_xdate()
plt.legend()
plt.title("Estimated active cases every day (all of Germany)")
plt.grid()
plt.show()

## Last seven days broken down by district

In [ ]:
start_at = pd.to_datetime(datetime.date.today() - datetime.timedelta(days=7))
lkr = rki_lkr[["county", "EWZ", "KFL"]]
sum_lkr = rki_cases[rki_cases.Meldedatum >= start_at].groupby("Landkreis").sum()[["cases", "deaths", "recovered"]]
counties = pd.merge(lkr, sum_lkr, right_on="Landkreis", left_on="county")
counties["cases/100000 inhabitants"] = (counties["cases"]/counties["EWZ"])*100000
counties["cases/km²"] = counties["cases"]/counties["KFL"]
counties["cases/inhabitants"] = counties["cases"]/counties["EWZ"]
counties["deaths/km²"] = counties["deaths"]/counties["KFL"]
counties["deaths/inhabitants"] = counties["deaths"]/counties["EWZ"]
counties["inhabitants"] = counties["EWZ"]
counties.drop(["EWZ", "KFL"], axis="columns", inplace=True)
counties.sort_values(by="cases/100000 inhabitants", ascending=False)

## Cases, recovered, deaths (all of Germany)